# Image Genenrative model by conditionals

## Image Generative Model
- 이미지를 생성할 수 있는 확률 모델링, p(x)를 구하고 싶다. 
- 되기만 한다면 다양한 활용 가능
 - image compression, image denoising, debluring, high resolution
- 근데 어렵다. 
 - 매우 고차원, 
   - p(x) : joint dist over 320*320 
 - 매우 구조적
- 기존 접근 
 - variation autoencoder 접근, but intracable 문제를 대처하기 위해 approximate inference 사용할 수 밖에 없다.
 
### SK-T brain의 Generative Model 101
 - [https://www.facebook.com/SKTBrain/posts/313726382331516](https://www.facebook.com/SKTBrain/posts/313726382331516)

## 좋은 Image generative model의 조건
- Expressive 
- ** Tractible **
- Scalable

## 오늘의 접근 방법 : factorization into conditionals
- joint를 conditionals 표현함으로서 tractible 하게 된다. => PGM의 접근
- p(x1,x2,x3) = p(x1) * p(x2|x1) * p(x3|x2)


# 번외 : RNN 변형 (방향성, 차원성, 계층성)
[http://www.slideshare.net/grigorysapunov/multidimensional-rnn](http://www.slideshare.net/grigorysapunov/multidimensional-rnn)

## 기본형 with LSTM

![](./images/pixelcnn/01.png)

## 방향성 변형

![](./images/pixelcnn/02.png)

## 차원 변형 (1-dim-> multi-dim)
- 기존 1-d만, 고차원 데이터(ex. images)에 부적합한 측면
- data에 ordering이 있는 것을 가정

![](./images/pixelcnn/03.png)
![](./images/pixelcnn/04.png)

## 방향성 + 차원 변형
- 다중 방향성, 다중 차원 
- 예시에서는 4방향, 2차원 

![](./images/pixelcnn/05.png)

## 계층성 (Hierachical subsampling RNN )

![](./images/pixelcnn/06.png)

---

# 첫번째 논문: Generative Image Modeling Using Spatial LSTMs
- [https://arxiv.org/abs/1506.03478](https://arxiv.org/abs/1506.03478)


## p(x) as factorized conditionals
- next pixel의 확률을 그 이전(위에서부터 행 by 행, 같은 행에서는 바로 그 pixel 직전까지) 것들에 종속되서 결정

![](./images/pixelcnn/07.png)

![](./images/pixelcnn/08.png)

## p(xi) as gaussian mixture models
- 각 p(xi)는 다양한 covariance matrix를 mixture로 묶은 모델

![](./images/pixelcnn/09.png)

## 현실적으로는 causal neighbor에만 종속되게
- next pixel의 그 이전 pixel을 모두 고려하면 너무 계산량과 parameter가 폭주
- markov 가정을 고려해서 최근 것만 고려
 - 삼각형 이웃 : 최근의 행, 최근의 열
 

## 2-D LSTM 을 deep하게 쌓은 학습 모델 구성
- 이러한 조건부 확률 분포들을 deep neural network를 통해 자동으로 학습하도록 함


---

# 두번째 논문 : Pixel Recurrent Neural Networks
- [https://arxiv.org/abs/1601.06759](https://arxiv.org/abs/1601.06759)


## 첫번째와 대동 소이 
- joint dist 을 조건부 확률 분포의 곱으로 표현 
- multi-dim LSTM 사용

## 개선점
- RGB 이미지로 확장 : R,G,B 끼리도 conditioning
- pdf 가 아닌 pmf : 256 value
- 12-depth 2-D LSTM
- 2방향 2-D LSTM


## RGB conditioning

![](./images/pixelcnn/10.png)
![](./images/pixelcnn/11.png)

## 256-value multinomial pmf

![](./images/pixelcnn/12.png)

## PixelRNN #1 : Row LSTM
- unidirectional
- 삼각형의 causal neighborhood

## PixelRNN #2 : Bidirectional 2-D LSTM

![](./images/pixelcnn/13.png)

## PixelRNN #3 : Multi-Scale 버젼
- 한 개의 subsampled pixelRNN
- 그 위에 여러 개의 up-sampled pixelRNN 쌓기
- 참고) deconv in CNN

![](./images/pixelcnn/14.png)

## 실험

![](./images/pixelcnn/15.png)
![](./images/pixelcnn/16.png)
![](./images/pixelcnn/17.png)

## 구현 : TensorFlow implementation of Pixel Recurrent Neural Networks
[https://github.com/carpedm20/pixel-rnn-tensorflow](https://github.com/carpedm20/pixel-rnn-tensorflow)




## 번외 : LSTM에서 착안한 gating 기법을 이용한 심층 네트워크 학습
- [https://arxiv.org/pdf/1507.06228v2.pdf](https://arxiv.org/pdf/1507.06228v2.pdf)



---

# 세번째 논문 :Conditional Image Generation with PixelCNN Decoder
- [https://arxiv.org/abs/1606.05328](https://arxiv.org/abs/1606.05328)


## PixcelCNN
- Input: N x N x 3 
- Output : N x N x 3 x 256
- Filter : 5 x 5 with mask

## Gated Convolutional Layers
- PixelRNN의 장점 분석
 - 각 층(layer)에서 previous pixel과의 풍부한 interaction이 있다. 
 - LSTM에서 multiplicative unit이 있다. => 비선형성? 좀 더 풍부한 복잡도(complexity)를 유발
- PixcelRNN에서 이러한 특징을 받아들이게 고치면
 - 층을 깊게 쌓자. 그러면 convolution의 특징상 층이 깊어지면 좀 더 많은 previous pixel과 interaction이 강해진다.
 - 기존 relu를 gated activation unit으로 바꾸어서 더 풍부한 복잡도를 갖게 하자.
 
![](./images/pixelcnn/19.png)

![](./images/pixelcnn/20.png)


## Conditional PixelCNN: 추가적인 정보에 dependent한 모델
- 외부에서 추가적인 정보를 제공할 때 generative model
 - ex) class label in CIFAR-10

![](./images/pixelcnn/18.png)

- additive 한 속성으로 보고,  y = Wx + Vh 처럼 모델링

![](./images/pixelcnn/21.png)
- 예를 들어 h가 class label 에 대한 one-hot encoding이라면 이는 마치 각 레이어별로 특정 클래스에 대한 선호(bias)가 있는 상황이 되는 것이다.
- 위의 식은 pixel location에 dependent하지 않다.
 - 왜냐하면 이미지의 어디에 대한 정보가 아니라 무엇에 대한 정보이기 때문이다.
- 이를 변형해서 위치 정보와 함께 제공할 수도 있다.
 - 이러면 특정 위치에서 유효한 정보를 주는 셈



## blind spot in receptive field
[TODO]


## PixelCNN AutoEncoder
[TODO]


## 실험

![](./images/pixelcnn/22.png)
![](./images/pixelcnn/23.png)
![](./images/pixelcnn/24.png)
![](./images/pixelcnn/25.png)
